#### 1. Create environment to run notebook (this allows to keep dependencies local and avoid conflicts)

```
python3 -m venv ./.venv
```
#### 2. Activate environment to install dependencies from terminal

```
source ./venv/bin/activate
```

#### 3. If you have not installed the notebook required packages, run the following command 


#### 4. If you use VS-Code - click Select Kernel, choose Python Environments, and select the environment you configured either directly invoking PIP3

```
pip3 install -r /Users/dac4/github/blossom-oscal/build/py/requirements.txt
```

#### or alternatively invoking PIP internal to Python 3

```
python3 -m pip install -r /Users/dac4/github/blossom-oscal/build/py/requirements.txt
```

#### Import Libraries


In [1]:
import os, sys
import chevron
import json
import datetime
from pathlib import Path
from yaml import safe_load,YAMLError,dump

from oscalic.system_security_plan import SystemSecurityPlan as SSP
from oscalic.control import ControlAssembly as Control
from oscalic import Template, Helper, Validation

error_condition = None

#### Setup tracking and context variables inside of the static Config

In [3]:
class Config:
    today_format = '%Y-%m-%dT00:00:00.0000-04:00'
    today = datetime.datetime.now().strftime(today_format)
    control_list = list()
    # Set Work-Paths e.g. build/py/system-security-plan/partials/template.ssp.yaml
    partials_path = 'build/py/system-security-plan/partials'
    # Read-In the Partials
    partials = os.listdir(partials_path)
    this_system_component_uuid = Helper.get_uuid()
    ssp_controls=list()

print(f'{len(Config.partials)=}')
_x = [print(f'\t{idx+1}.\t{part}') for idx, part in enumerate(Config.partials)]

len(Config.partials)=47
	1.	ra-5.11.yaml
	2.	template.profile.yaml
	3.	ac-17.2.yaml
	4.	template.ssp.yaml
	5.	sc-7.5.yaml
	6.	template.yaml
	7.	au-3.1.yaml
	8.	ac-2.1.yaml
	9.	ac-6.7.yaml
	10.	sc-7.4.yaml
	11.	ac-17.3.yaml
	12.	sc-5.yaml
	13.	sc-7.8.yaml
	14.	ra-5.yaml
	15.	sc-7.3.yaml
	16.	ac-6.yaml
	17.	sc-2.yaml
	18.	ia-2.1.yaml
	19.	ia-2.12.yaml
	20.	ac-17.4.yaml
	21.	ac-7.yaml
	22.	ac-6.1.yaml
	23.	au-12.yaml
	24.	ac-2.5.yaml
	25.	ac-6.2.yaml
	26.	ac-17.yaml
	27.	ac-8.yaml
	28.	ia-2.2.yaml
	29.	ac-2.4.yaml
	30.	ra-5.2.yaml
	31.	ac-2.13.yaml
	32.	ac-2.yaml
	33.	au-3.yaml
	34.	ra-5.5.yaml
	35.	ac-2.3.yaml
	36.	sc-7.7.yaml
	37.	ia-2.yaml
	38.	ia-2.8.yaml
	39.	ac-6.5.yaml
	40.	ac-2.2.yaml
	41.	au-2.yaml
	42.	sa-11.yaml
	43.	ac-3.yaml
	44.	ac-6.10.yaml
	45.	sc-7.yaml
	46.	ac-6.9.yaml
	47.	ac-17.1.yaml


In [16]:
## Build the path to the SSP Template
ssp_template = os.path.join(os.getcwd(), Config.partials_path, 'template.ssp.yaml')
## Prepare the 
ssp_data = {
    'uuid:document':        Helper.get_uuid(),
    'uuid:statement':       Helper.get_uuid(),
    'uuid:component':       Config.this_system_component_uuid, 
    'uuid:user':            Helper.get_uuid(),
    'uuid:party':           Helper.get_uuid(), 
    'uuid:by-component':    Helper.get_uuid(), 
    'uuid:information-type':Helper.get_uuid(), 
    'version':              '0.0.1',
    'modified_date':        f"{Config.today}",
}
ssp_content = Template.apply(ssp_template, ssp_data)
print(ssp_content)
ssp = Helper.from_yaml(SSP, ssp_content)

system-security-plan:
  uuid: 511a5338-ef61-4f03-8fc1-536bc1287887

  metadata:
    title: BloSS@M Experimental System Security Plan
    last-modified: '2022-12-21T00:00:00.0000-04:00'
    version: 0.0.1
    oscal-version: 1.0.4
    roles:
      - id: admin
        title: Administrator
    parties:
      - uuid: 096c1453-5bef-438f-8109-157c7d3ee88b
        type: person
  
  import-profile:
    href: ./profile.yaml
  
  system-characteristics:
    system-ids:
      - id: saas_system_iaas_customer
    system-name: Leveraging SaaS System
    description: >
      NO CONTENT HERE FOR NOW
    security-sensitivity-level: low
    system-information:
      information-types:
        - uuid: 7e84a805-33da-4c5c-bac1-c25c6d49bafd
          title: System Assessment
          description: This system handles development information pertaining to audit and assessment events to demonstrate the OSCAL workflow.
          categorizations:
            - system: https://doi.org/10.6028/NIST.SP.800-60v2r1
 

ValidationError: 1 validation error for SystemSecurityPlan
system-security-plan.controls
  Field required [type=missing, input_value={'uuid': '511a5338-ef61-4...nted-requirements': []}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing

In [8]:
# Profile build-up beginning

profile_template = os.path.join(os.getcwd(), Config.partials_path, 'template.profile.yaml')
profile_data = {
    'uuid:document':        Helper.get_uuid(),
    'uuid:statement':       Helper.get_uuid(),
    'uuid:component-uuid':  Config.this_system_component_uuid, 
    'uuid:by-component':    Helper.get_uuid(),
    'version':              '0.0.1',
    'modified_date':        f"{Config.today}"
}


In [10]:
for partial in Config.partials:
    if partial.startswith('template.'):
        continue

    partial_file = os.path.join(os.getcwd(), Config.partials_path, partial)

    uuid_content = {
        'uuid:control':         Helper.get_uuid(),
        'uuid:statement':       Helper.get_uuid(),
        'uuid:component-uuid':  Config.this_system_component_uuid, 
        'uuid:by-component':    Helper.get_uuid(), 
    }

    partial_content = Template.apply(partial_file, uuid_content)

    try:
        control = Helper.from_yaml(Control, partial_content)
        ssp.system_security_plan.control_implementation.implemented_requirements.append(control)
        print(f"SUCCESS: {partial_file}")
    except Validation.OSCALValidationError as e:
        print(f"{partial_file}:\nVALIDATION ERROR: {e.json()}\n")
        error_condition = 1


NameError: name 'ssp' is not defined

In [ ]:
###################################################################################################
## Prepare Document
profile_content = Template.apply(profile_template, profile_data)
Path('Profile.output.yaml').write_text(profile_content)

#%% Save SSP
Path('SSP.output.yaml').write_text(Helper.to_yaml(ssp))

# %%
if error_condition:
    exit(error_condition)

In [19]:
x = {
    'metadata':
        {   #......
            'link': {'@href':'./blossom_moderate_profile.xml',
                     '@rel':'source-profile'
                     },
            'link': {'@href':'./other-blossom_moderate_profile.xml',
                     '@rel':'source-profile'
                     }
         }    
}

print(f'{x['metadata']['link']['@href']=}')

x['metadata']['link']['@href']='./some-new-blossom_moderate_profile.xml'
# XPath '/metadata/link[last]/@href'

print(f'{x['metadata']['link']['@href']=}')

x['metadata']['link']['@href']='./other-blossom_moderate_profile.xml'
x['metadata']['link']['@href']='./some-new-blossom_moderate_profile.xml'
